In [2]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import pyttsx3

# Load the model
model = load_model(r"D:/python/SignLanguageRecognition/model/asl_model.h5")  # Path to your trained model
label_map = list("0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ")

# Text-to-speech
engine = pyttsx3.init()
engine.setProperty('rate', 150)
engine.setProperty('volume', 1.0)

# MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Video Capture
cap = cv2.VideoCapture(0)

current_prediction = ""
last_spoken = ""

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:
        x_vals, y_vals = [], []

        for hand_landmarks in result.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                x_vals.append(int(lm.x * w))
                y_vals.append(int(lm.y * h))

            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Combined bounding box for both hands
        xmin = max(min(x_vals) - 20, 0)
        xmax = min(max(x_vals) + 20, w)
        ymin = max(min(y_vals) - 20, 0)
        ymax = min(max(y_vals) + 20, h)

        roi = frame[ymin:ymax, xmin:xmax]
        try:
            img = cv2.resize(roi, (64, 64)) / 255.0
            img = img.reshape(1, 64, 64, 3)

            preds = model.predict(img)
            label = label_map[np.argmax(preds)]
            current_prediction = label

            if current_prediction != last_spoken:
                engine.say(current_prediction)
                engine.runAndWait()
                last_spoken = current_prediction

        except Exception as e:
            print("Error in processing frame:", e)

    # Display
    cv2.putText(frame, f"Prediction: {current_prediction}", (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Sign Language Detection (Two Hands)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

